### MobileNet
- Neural network which is around 28MB. 
- Runs on mobile devices.
- Single-shot detection.

### Bring in an image to this model (do inference on a trained model)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

C:\Users\pablo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
 model = MobileNet() # weights="path to the h5 file"

In [3]:
img = image.load_img('./data/hotdog/train/hot_dog/1017226.jpg', target_size=(224,224))
img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
img_array.shape

(224, 224, 3)

In [4]:
probas = model.predict(img_array.reshape(1, 224,224,3))

1/1 [==============================] - 1s 603ms/step


In [5]:
decode_predictions(probas)

35363/35363 [==============================] - 0s 2us/step


[[('n02782093', 'balloon', 0.4228459),
  ('n09256479', 'coral_reef', 0.10597261),
  ('n02606052', 'rock_beauty', 0.08158792),
  ('n04209239', 'shower_curtain', 0.080399424),
  ('n03530642', 'honeycomb', 0.04067596)]]

#### Load as a json file (if no access to internet)

In [6]:
#import json
#with open("json file path") as f:
#    imagenet = json.load(f)
# probas

## Transfer learning

- Bring back `MobileNet` and do not include the top layer on the network (the last layer excluded).

In [7]:
base_model = MobileNet()

#### Take the output of the last but one layer, and use it to build on top of it.

In [8]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, Sequential

In [9]:
len(base_model.layers)

91

In [11]:
new_model = Sequential([base_model.layers[i] for i in range(91)])
new_model.trainable = False  # True = We do fine-tuning, False = Feature extraction + fine-tuning comes later
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(2, activation='softmax'))

In [12]:
#new_new_model = Sequential(base_model.layers[::-1]+[Dense(512, activation='relu'), Dense(2, activation='softmax')])

#### Use generator to bring in the training images for hotdog/not hotdog 

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   , horizontal_flip=True, vertical_flip=True)

In [15]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   , horizontal_flip=True, vertical_flip=True)

In [16]:
train_generator = train_datagen.flow_from_directory('./data/hotdog/train/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    class_mode='categorical', 
                                                    batch_size=16)

Found 498 images belonging to 2 classes.


In [17]:
test_generator = test_datagen.flow_from_directory('./data/hotdog/test/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    class_mode='categorical', 
                                                    batch_size=16)

Found 500 images belonging to 2 classes.


In [18]:
new_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
history = new_model.fit_generator(generator=train_generator,
                                  epochs=20,
                                  validation_data=test_generator
                                 )

Epoch 1/20


C:\Users\pablo\AppData\Local\Temp\ipykernel_21852\825767053.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(generator=train_generator,




32/32 [==============================] - 10s 268ms/step - loss: 0.6910 - accuracy: 0.5582 - val_loss: 0.6912 - val_accuracy: 0.5660
Epoch 2/20
32/32 [==============================] - 5s 164ms/step - loss: 0.6911 - accuracy: 0.5422 - val_loss: 0.6910 - val_accuracy: 0.5680
Epoch 3/20
32/32 [==============================] - 5s 156ms/step - loss: 0.6908 - accuracy: 0.5562 - val_loss: 0.6906 - val_accuracy: 0.5840
Epoch 4/20
32/32 [==============================] - 5s 164ms/step - loss: 0.6906 - accuracy: 0.5703 - val_loss: 0.6911 - val_accuracy: 0.5640
Epoch 5/20
32/32 [==============================] - 5s 157ms/step - loss: 0.6915 - accuracy: 0.5542 - val_loss: 0.6907 - val_accuracy: 0.5640
Epoch 6/20
32/32 [==============================] - 5s 159ms/step - loss: 0.6913 - accuracy: 0.5502 - val_loss: 0.6906 - val_accuracy: 0.5780
Epoch 7/20
32/32 [==============================] - 5s 162ms/step - loss: 0.6910 - accuracy: 0.5482 - val_loss: 0.6911 - val_accuracy: 0.5560
Epoch 8/20
32/

In [20]:
def load_img(file):
    img = image.load_img(file, target_size=(224,224))
    img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
    return img_array.reshape(1, 224,224,3)

In [21]:
preds_to_labels = { train_generator.class_indices[k]:k for k in train_generator.class_indices.keys()}

In [22]:
counter = {'hot_dog':0, 'not_hot_dog':0}
import glob
for file in glob.glob('./data/hotdog/test/hot_dog/*'):
    preds = new_model.predict(load_img(file))
    pred = np.argmax(preds)
    counter[preds_to_labels[pred]] += 1

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 38ms/step


In [23]:
counter

{'hot_dog': 248, 'not_hot_dog': 2}

## Model persistance

In [24]:
import tensorflow as tf

In [ ]:
tf.saved_model.save(new_model, "./tmp/mobilenet/1/")